# template から HTML と JS を生成する


In [40]:
# Move up to top directory
import os
from pathlib import Path

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir("../")

print(os.getcwd())

/mnt/e/TSUMUGI
/mnt/e/TSUMUGI


# 古いファイルの消去


In [41]:
%%bash

rm -rf web/network/phenotype
rm -rf web/network/genesymbol
rm -rf web/network/data

mkdir -p web/network/phenotype
mkdir -p web/network/genesymbol
mkdir -p web/network/data

echo "*" > web/network/phenotype/.gitignore
echo "*" > web/network/genesymbol/.gitignore
echo "*" > web/network/data/.gitignore


## index.htmlの描画に必要なファイルを生成する


In [42]:
mp_terms = {}
for path_mp_term in Path("data", "mp_term_name").glob("*.csv"):
    mp_term = path_mp_term.stem
    # print(path_mp_term.stem)
    if not Path("data", "network", f"{mp_term}.json").exists():
        continue
    mp_term_name_space = mp_term.replace("_", " ")
    # print(mp_term_name_space)
    mp_terms[mp_term_name_space] = mp_term

In [43]:
print(list(mp_terms)[:3])
print(len(mp_terms))

['abnormal abdominal wall morphology', 'abnormal adrenal gland morphology', 'abnormal allantois morphology']
535


In [44]:
import json

json.dump(mp_terms, open("data/available_mp_terms.json", "w"), indent=2)

In [45]:
%%bash

cp data/available_mp_terms.json web/network/data/available_mp_terms.json
cp data/available_gene_symbols.txt web/network/data/available_gene_symbols.txt
cp data/marker_symbol_accession_id.json web/network/data/marker_symbol_accession_id.json


## index.htmlを生成

In [46]:
%%bash
grep -v "REMOVE_THIS_LINE" web/network/template/template_index.html > web/index.html

# JSON のコピー


In [47]:
%%bash

rm -rf web/network/phenotype/data
rm -rf web/network/genesymbol/data

mkdir -p web/network/phenotype/data
mkdir -p web/network/genesymbol/data

cp -r data/network/mp_term_name/* web/network/phenotype/data
cp -r data/network/gene_symbol/* web/network/genesymbol/data

# 1 min 18 sec

# Template から HTML と JS を生成


In [48]:
%%bash

########################################
# Phenotype
########################################
mkdir -p web/network/phenotype/js

find data/mp_term_name -type f |
sed "s|data/mp_term_name/||" |
sed "s|.csv$||" |
while read mp_term_name_underscore; do
    mp_term_name_space=$(echo "$mp_term_name_underscore" | sed "s|_| |g")
    impc_url=$(awk -F "\t" -v mp="$mp_term_name_space" '$1 == mp {print $2}' data/mp_term_name_id_url.tsv)
    escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')
    # HTML
    cat web/network/template/template_network_phenotype.html |
    grep -v "REMOVE_THIS_LINE" |
    sed "s|XXX_impc_url|${escaped_url}|g" |
    sed "s|XXX_mp_term_name_space|${mp_term_name_space}|g" |
    sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > web/network/phenotype/"$mp_term_name_underscore".html

    # Javascript
    cat web/network/template/template_network_phenotype.js |
    grep -v "REMOVE_THIS_LINE" |
    sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > web/network/phenotype/js/"$mp_term_name_underscore".js
done
# 24.4s

In [49]:
%%bash

########################################
# Gene Symbol
########################################
mkdir -p web/network/genesymbol/js

cat data/available_gene_symbols.txt |
while read gene_symbol; do
    impc_url="https://www.mousephenotype.org/data/genes/"$(awk -F "\t" -v key="$gene_symbol" '$1 == key {print $2}' data/marker_symbol_accession_id.tsv)
    escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')
    # HTML
    cat web/network/template/template_network_genesymbol.html |
    grep -v "REMOVE_THIS_LINE" |
    sed "s|XXX_impc_url|${escaped_url}|g" |
    sed "s|XXX_genesymbol|${gene_symbol}|g" > web/network/genesymbol/"$gene_symbol".html

    # Javascript
    cat web/network/template/template_network_genesymbol.js |
    grep -v "REMOVE_THIS_LINE" |
    sed "s|XXX_genesymbol|${gene_symbol}|g" > web/network/genesymbol/js/"$gene_symbol".js
done

# 5 min

## テスト用のデータをtest-tsumugiにコピー

In [50]:
%%bash

rm -rf test-tsumugi
mkdir -p test-tsumugi/network/
echo "*" > test-tsumugi/.gitignore

mkdir -p test-tsumugi/network/phenotype
mkdir -p test-tsumugi/network/genesymbol
(cd test-tsumugi/network/phenotype && mkdir -p data js)
(cd test-tsumugi/network/genesymbol && mkdir -p data js)

In [51]:
%%bash

cp -r web/index.html test-tsumugi/
cp -r web/image test-tsumugi/
cp -r web/network/css test-tsumugi/network/
cp -r web/network/data test-tsumugi/network

In [52]:
%%bash
# Phenotype: male_infertility
find web/network/phenotype/ -type f |
    grep /male_infertility |
    while read path_input; do
        path_output=$(echo $path_input | sed "s|web/|test-tsumugi/|")
        echo $path_output
        cp $path_input $path_output
    done

test-tsumugi/network/phenotype/data/male_infertility.json


test-tsumugi/network/phenotype/js/male_infertility.js
test-tsumugi/network/phenotype/male_infertility.html


In [53]:
%%bash
# Gene: Rab10
find web/network/genesymbol// -type f |
    grep /Rab10 |
    while read path_input; do
        path_output=$(echo $path_input | sed "s|web/|test-tsumugi/|")
        echo $path_output
        cp $path_input $path_output
    done

test-tsumugi/network/genesymbol//data/Rab10.json
test-tsumugi/network/genesymbol//js/Rab10.js
test-tsumugi/network/genesymbol//Rab10.html
